# GCC118 - Programação Matemática
## Universidade Federal de Lavras
### Instituto de Ciências Exatas e Tecnológicas
#### Aluno: Marcos Carvalho Ferreira

In [ ]:
!pip install pulp
import json
from pathlib import Path
!pip install pandas
import pandas as pd
import pulp

Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 1.6 MB/s eta 0:00:001.9 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.5 MB/s eta 0:00:00
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 4.5 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 4.7 MB/s eta 0:00:00m eta 0:00:010:00:01
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
Note: you may need to restart the kernel to use updated packages.


## Problema 4

Uma nutricionista deseja formular uma dieta de emagrecimento, de forma a atender às necessidades mı́nimas de nutrientes de um indivı́duo, respeitando limites calóricos e de custo, e controlando o excesso de alguns nutrientes especı́ficos. Você dispõe de uma base de dados contendo 30 alimentos, cada um com informações de:

* preço por porção (R$),
* valor energético (kcal),
* proteı́na (g),
* gordura (g),
* carboidrato (g),
* cálcio (mg),
* ferro (mg),
* vitamina C (mg),
* sódio (mg).

As informações estão armazenadas em um arquivo JSON no seguinte formato simplificado:

```json
{
  "params": {
    "Kmax": 1600,
    "Cmax": 20,
    "min_prot": 60,
    "min_carb": 130,
    "min_cal": 1000,
    "min_fe": 8,
    "min_vitc": 75,
    "fat_max": 50,
    "sod_max": 2300
  },
  "alimentos": [
    { "id": 1, "nome": "Arroz integral", ... },
    { "id": 2, "nome": "Feijão carioca", ... },
    ...
    { "id": 30, "nome": "Sardinha enlatada", ... }
  ]
}
```

### Regras

A dieta diária deve conter no máximo 1600 kcal e no máximo R\$ 20,00.

  * Deve-se garantir um consumo mı́nimo de:
    * 60 g de proteı́na,
    * 130 g de carboidratos,
    * 1000 mg de cálcio,
    * 8 mg de ferro,
    * 75 mg de vitamina C.
  * O consumo de gordura deve ser no máximo 50 g/dia.
  * O consumo de sódio deve ser no máximo 2300 mg/dia.

O objetivo não é minimizar o custo da dieta, mas sim minimizar a quantidade de
nutrientes que ultrapassam os limites máximos permitidos. Formule seu problema
por meio de um PL, e resolva-o.

## Parâmetros

* $K_{max}$ = Limite máximo de energia: 1600 kcal
* $C_{max}$ = Limite máximo de custo: R\$20  
* $P_{min}$ = Mínimo de proteína 60 g
* $CH_{min}$ = Mínimo de carboidrato 130 g
* $Ca_{min}$ = Mínimo de cálcio 100 mg
* $Fe_{min}$ = Mínimo de ferro 8mg
* $VitC_{min}$ = Mínimo de vitamina C 75 mg
* $G_{max}$ = Limite máximo de gordura 50 g
* $Na_{max}$ = Limite máximo de sódio 2300 mg
* $kcal_i, c_i, p_i, g_i, ch_i, ca_i, fe_i, vitc_i, na_i$ = Valores nutricionais e custo por porção do alimento $i$

## Variáveis de decisão

* $x_i \ge 0$: quantidade de porções do alimento $i$.
* $e_{kcal}, e_{custo}, e_{gord}, e_{sod} \ge 0$: variáveis de excesso para permitir ultrapassar os limites máximos de energia, custo, gordura e sódio.


In [3]:
path = Path("dieta.json")
data = json.loads(path.read_text())

params = data["params"]
alimentos = data["alimentos"]

ids = [alimento["id"] for alimento in alimentos]
nome = {alimento["id"]: alimento["nome"] for alimento in alimentos}
price = {alimento["id"]: alimento["preco"] for alimento in alimentos}
kcal = {alimento["id"]: alimento["kcal"] for alimento in alimentos}
prot = {alimento["id"]: alimento["proteina"] for alimento in alimentos}
fat = {alimento["id"]: alimento["gordura"] for alimento in alimentos}
carb = {alimento["id"]: alimento["carboidrato"] for alimento in alimentos}
calcio = {alimento["id"]: alimento["calcio"] for alimento in alimentos}
ferro = {alimento["id"]: alimento["ferro"] for alimento in alimentos}
vitc = {alimento["id"]: alimento["vitaminaC"] for alimento in alimentos}
sodio = {alimento["id"]: alimento["sodio"] for alimento in alimentos}

## Modelo matemático

### Função Objetivo

Minimizar a soma dos excessos.

$$
\min Z = e_{kcal} + e_{custo} + e_{gord} + e_{sod}
$$


### Restrições

1. Garantir que a dieta não ultrapasse os valores máximos de energia, custo, gordura e sódio. Se ultrapassar, o excesso será contabilizado nas variáveis
$e$:

$$
\begin{aligned}
\sum_{i} kcal_i \, x_i - e_{kcal} &\le K_{max} \\
\sum_{i} c_i \, x_i - e_{custo} &\le C_{max} \\
\sum_{i} g_i \, x_i - e_{gord} &\le G_{max} \\
\sum_{i} na_i \, x_i - e_{sod} &\le Na_{max}
\end{aligned}
$$

2. A escolha das porções devem respeitar os limites mínimos dos nutrientes obrigatórios:

$$
\begin{aligned}
\sum_{i} p_i \, x_i &\ge P_{min} \\
\sum_{i} ch_i \, x_i &\ge CH_{min} \\
\sum_{i} ca_i \, x_i &\ge Ca_{min} \\
\sum_{i} fe_i \, x_i &\ge Fe_{min} \\
\sum_{i} vitc_i \, x_i &\ge VitC_{min}
\end{aligned}
$$

 3. Domínio:

$$
x_i \ge 0 \quad \forall i, \qquad
e_{kcal}, e_{custo}, e_{gord}, e_{sod} \ge 0
$$


### Declaração do objeto que representa o modelo matemático

In [4]:
modelo = pulp.LpProblem("Dieta", pulp.LpMinimize)

### Variáveis de Decisão

In [5]:
# Variáveis de decisão (porções)
x = {i: pulp.LpVariable(f"x_{i}", lowBound=0, cat='Integer') for i in ids}

# Variáveis de excesso
e_kcal  = pulp.LpVariable("e_kcal",  lowBound=0, cat='Integer')
e_custo  = pulp.LpVariable("e_custo",  lowBound=0, cat='Integer')
e_gord   = pulp.LpVariable("e_gord",   lowBound=0, cat='Integer')
e_sod   = pulp.LpVariable("e_sod",   lowBound=0, cat='Integer')

### Função Objetivo

In [6]:
modelo += e_kcal + e_custo + e_gord + e_sod, "Minimizar_excessos"

### Restrições

In [7]:
modelo += pulp.lpSum(kcal[i]*x[i] for i in ids) - e_kcal <= params["Kmax"]
modelo += pulp.lpSum(price[i]*x[i] for i in ids) - e_custo <= params["Cmax"]
modelo += pulp.lpSum(fat[i]*x[i] for i in ids) - e_gord <= params["fat_max"]
modelo += pulp.lpSum(sodio[i]*x[i] for i in ids) - e_sod <= params["sod_max"]

modelo += pulp.lpSum(prot[i]*x[i] for i in ids) >= params["min_prot"]
modelo += pulp.lpSum(carb[i]*x[i] for i in ids) >= params["min_carb"]
modelo += pulp.lpSum(calcio[i]*x[i] for i in ids) >= params["min_cal"]
modelo += pulp.lpSum(ferro[i]*x[i] for i in ids) >= params["min_fe"]
modelo += pulp.lpSum(vitc[i]*x[i] for i in ids) >= params["min_vitc"]

In [8]:
modelo

Dieta:
MINIMIZE
1*e_custo + 1*e_gord + 1*e_kcal + 1*e_sod + 0
SUBJECT TO
_C1: - e_kcal + 130 x_1 + 180 x_10 + 100 x_11 + 200 x_12 + 25 x_13 + 15 x_14
 + 40 x_15 + 50 x_16 + 90 x_17 + 80 x_18 + 70 x_19 + 120 x_2 + 85 x_20
 + 160 x_21 + 200 x_22 + 190 x_23 + 110 x_24 + 115 x_25 + 150 x_26 + 100 x_27
 + 90 x_28 + 80 x_29 + 150 x_3 + 120 x_30 + 165 x_4 + 70 x_5 + 95 x_6
 + 110 x_7 + 250 x_8 + 120 x_9 <= 1600

_C2: - e_custo + 0.5 x_1 + 1.8 x_10 + 0.8 x_11 + 0.9 x_12 + 0.5 x_13
 + 0.4 x_14 + 0.6 x_15 + 0.8 x_16 + 0.5 x_17 + 0.7 x_18 + 0.6 x_19 + 0.7 x_2
 + 0.8 x_20 + 1.2 x_21 + 1.5 x_22 + x_23 + 0.8 x_24 + 0.9 x_25 + 0.7 x_26
 + 0.6 x_27 + x_28 + 0.7 x_29 + 0.4 x_3 + 0.9 x_30 + 2 x_4 + 0.6 x_5 + 1.2 x_6
 + 1.5 x_7 + 2.5 x_8 + 2.2 x_9 <= 20

_C3: - e_gord + x_1 + 10 x_10 + 0.1 x_11 + 1.5 x_12 + 0.2 x_13 + 0.1 x_14
 + 0.2 x_15 + 0.5 x_16 + 0.3 x_17 + 0.2 x_18 + 0.1 x_19 + 0.5 x_2 + 0.3 x_20
 + 15 x_21 + 18 x_22 + 16 x_23 + 0.5 x_24 + 0.4 x_25 + 3 x_26 + 0.5 x_27
 + 10 x_28 + 9 x_29 + 2 x_3 + 

### Resolvendo o problema

In [9]:
modelo.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/marcos/Área de trabalho/Mestrado/1 - Disciplina Isolada/Programação Matemática/Atividades Práticas/Atividade 1/atividade1/.venv/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/ff2162aa2e494e67a01f4fc3981666fd-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/ff2162aa2e494e67a01f4fc3981666fd-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 14 COLUMNS
At line 316 RHS
At line 326 BOUNDS
At line 361 ENDATA
Problem MODEL has 9 rows, 34 columns and 229 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0003I 0 fixed, 32 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0004I processed model has 9 rows, 32 columns (32 integer (0 of which binary)) and 222 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0012I I

1

## Imprimindo as soluções do problema

In [10]:
print("Status:", pulp.LpStatus[modelo.status])
print("Objetivo (soma dos excessos):", pulp.value(modelo.objective))

# Totais
tot_price = sum(price[i]*x[i].value() for i in ids)
tot_kcal  = sum(kcal[i]*x[i].value() for i in ids)
tot_prot  = sum(prot[i]*x[i].value() for i in ids)
tot_carb  = sum(carb[i]*x[i].value() for i in ids)
tot_calcio= sum(calcio[i]*x[i].value() for i in ids)
tot_ferro = sum(ferro[i]*x[i].value() for i in ids)
tot_vitc  = sum(vitc[i]*x[i].value() for i in ids)

print(f"Custo total = R$ {tot_price:.2f}")
print(f"Energia total = {tot_kcal:.1f} kcal")
print(f"Proteína total = {tot_prot:.1f} g")
print(f"Carboidratos = {tot_carb:.1f} g")
print(f"Cálcio = {tot_calcio:.1f} mg")
print(f"Ferro = {tot_ferro:.2f} mg")
print(f"Vitamina C = {tot_vitc:.1f} mg")

print("\nExcessos:")
print(" e_kcal=", e_kcal.value(), " e_custo=", e_custo.value(), " e_gord=", e_gord.value(), " e_sod=", e_sod.value())

# Tabela de porções escolhidas
sol = []
for i in ids:
    q = x[i].value()
    if q is not None and q > 1e-8:
        sol.append({
            "nome": nome[i],
            "porcoes": round(q,4),
            "preco_un": price[i],
            "kcal_un": kcal[i],
            "prot_un": prot[i],
            "gord_un": fat[i],
            "carb_un": carb[i],
            "calcio_un": calcio[i],
            "ferro_un": ferro[i],
            "vitc_un": vitc[i],
            "sodio_un": sodio[i]
        })

df = pd.DataFrame(sol).sort_values("porcoes", ascending=False).reset_index(drop=True)
print("\nPorções selecionadas:")
print(df.to_string(index=False))

Status: Optimal
Objetivo (soma dos excessos): 0.0
Custo total = R$ 9.80
Energia total = 1080.0 kcal
Proteína total = 70.5 g
Carboidratos = 142.0 g
Cálcio = 1045.0 mg
Ferro = 10.50 mg
Vitamina C = 124.0 mg

Excessos:
 e_kcal= 0.0  e_custo= 0.0  e_gord= 0.0  e_sod= 0.0

Porções selecionadas:
             nome  porcoes  preco_un  kcal_un  prot_un  gord_un  carb_un  calcio_un  ferro_un  vitc_un  sodio_un
     Feijão preto      3.0       0.8      110      9.0      0.5       19         45       2.5        0         0
  Iogurte natural      2.0       1.2       95      6.0      3.0       10        150       0.1        2         2
          Laranja      2.0       0.6       70      1.0      0.1       18         40       0.2       60         0
   Arroz integral      1.0       0.5      130      2.5      1.0       28         10       0.2        0         0
Sardinha enlatada      1.0       1.8      180     20.0     10.0        0        300       2.0        0         0
     Queijo Minas      1.0     